## Implémentation de l'exemple 1 de la partie 1 du projet

Dans ce qui suit nous allons calculer à l'aide du solveur Gurobi, la solution optimale du programme linéarisé en variables mixtes de l'exemple 1 - Partie 1 MOGPL

A noter qu'aucune instruction spécifique n'est nécessaire quant à l'exécution du programme. Toutes les données sont préalablement déclarées et les sorties bien explicites et expliquées 

In [4]:
import gurobipy as gp
import numpy as np
from gurobipy import *

#Commençons par déclarer nos données 
u = [[5,6,4,8,1],[3,8,6,2,5]] #L'utilité des agents 1 et 2
W = [1,1] #W'k 
n = 2 #Le nombre d'agents 
p = 5 #Le nombre d'objets disponibles 

model = gp.Model('Partie 1 - Projet MOGPL')

#Déclaration de nos variables de décision 
x = np.array(
    [model.addVar(vtype = GRB.BINARY, name="x%d" %(i+1)) for i in range(p)])

z = np.array(
    [model.addVar(vtype = GRB.CONTINUOUS, lb = 0, name="z%d" %(i+1)) for i in range(n)])

r = np.array(
    [model.addVar(vtype = GRB.CONTINUOUS, lb = - GRB.INFINITY, name = 'r%d' %(k+1)) for k in range(n)])

b = np.array([
    [model.addVar(vtype = GRB.CONTINUOUS, lb = 0, name="b%d_%d" % (i+1,k+1)) for k in range(n)] for i in range(n)
])

#Déclarations des contraintes :
for i in range(n):
        model.addConstr( z[i] == quicksum(u[i][j]* x[j] for j in range(p)))

for k in range(n):
        for i in range(n):    
            model.addConstr( r[k] - b[i][k] <= z[i])

model.addConstr( quicksum(x[j] for j in range(p)) == 3)

# maj du modele pour integrer les nouvelles variables
model.update()

# definition de l'objectif
fonction_obj = LinExpr();
fonction_obj =0
for k in range(n):
        fonction_obj += W[k]*( (k+1)*r[k] - quicksum(b[i][k] for i in range(n)) )

model.setObjective(fonction_obj,GRB.MAXIMIZE)

# Resolution
model.optimize()

xfinal=np.array([x[j].x  for j in range(p)]) #Pour récupérer la valeur optimale de x
zfinal=np.array([z[i].x for i in range(n)]) #Pour récupérer la valeur optimale de z
    


print('La solution optimale du problème est (Les objets à prendre sont :)',xfinal)
print('La valeur optimale du problème est (la satisfaction max des deux agents respectivement): ', zfinal)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 13 columns and 29 nonzeros
Model fingerprint: 0xb3de01b2
Variable types: 8 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 3e+00]
Presolve removed 3 rows and 4 columns
Presolve time: 0.01s
Presolved: 4 rows, 9 columns, 19 nonzeros
Variable types: 0 continuous, 9 integer (5 binary)
Found heuristic solution: objective 39.0000000
Found heuristic solution: objective 47.0000000

Root relaxation: objective 5.000000e+01, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      50.0000000   50.00